# Loading the data

Import needed libraries

In [1]:
import pandas as pd
import torch
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import os

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE

'cuda'

Setup global variables for loading

In [2]:
# when working locally set correct paths from the current directory to

# directory that contains data from kaggle hms
INPUT_DATA_DIR = "data"

# directory in which our npy files are/will be stored
PROCESSED_DATA_DIR = "processed_data"

Load the metadata

In [3]:
train_meta_full = pd.read_csv(INPUT_DATA_DIR + "/train.csv")
train_meta = train_meta_full.loc[train_meta_full["eeg_sub_id"] == 0]
test_meta = pd.read_csv(INPUT_DATA_DIR + "/test.csv")

Create a butterfilter

In [4]:
from scipy.signal import butter, lfilter

def butter_lowpass_filter(data, cutoff_freq=20, sampling_rate=200, order=4):
    nyquist = 0.5 * sampling_rate
    normal_cutoff = cutoff_freq / nyquist
    b, a = butter(order, normal_cutoff, btype="low", analog=False)
    filtered_data = lfilter(b, a, data, axis=0)
    return filtered_data

Extract parquet data

In [5]:
# take a parquet dataframe and compute correct values for each column
# we want columns such as "Fp1-F7" as can be seen in /example_figures
def extract_parquet(parquet_data: torch.tensor):
    parquet_data["Fp1-F7"] = parquet_data["Fp1"] - parquet_data["F7"]
    parquet_data["F7-T3"] = parquet_data["F7"] - parquet_data["T3"]
    parquet_data["T3-T5"] = parquet_data["T3"] - parquet_data["T5"]
    parquet_data["T5-O1"] = parquet_data["T5"] - parquet_data["O1"]

    parquet_data["Fp2-F8"] = parquet_data["Fp2"] - parquet_data["F8"]
    parquet_data["F8-T4"] = parquet_data["F8"] - parquet_data["T4"]
    parquet_data["T4-T6"] = parquet_data["T4"] - parquet_data["T6"]
    parquet_data["T6-O2"] = parquet_data["T6"] - parquet_data["O2"]

    parquet_data["Fp1-F3"] = parquet_data["Fp1"] - parquet_data["F3"]
    parquet_data["F3-C3"] = parquet_data["F3"] - parquet_data["C3"]
    parquet_data["C3-P3"] = parquet_data["C3"] - parquet_data["P3"]
    parquet_data["P3-O1"] = parquet_data["P3"] - parquet_data["O1"]

    parquet_data["Fp2-F4"] = parquet_data["Fp2"] - parquet_data["F4"]
    parquet_data["F4-C4"] = parquet_data["F4"] - parquet_data["C4"]
    parquet_data["C4-P4"] = parquet_data["C4"] - parquet_data["P4"]
    parquet_data["P4-O2"] = parquet_data["P4"] - parquet_data["O2"]

    parquet_data["Fz-Cz"] = parquet_data["Fz"] - parquet_data["Cz"]
    parquet_data["Cz-Pz"] = parquet_data["Cz"] - parquet_data["Pz"]

    parquet_data = parquet_data.drop(
        [
            "Fp1",
            "F3",
            "C3",
            "P3",
            "F7",
            "T3",
            "T5",
            "O1",
            "Fz",
            "Cz",
            "Pz",
            "Fp2",
            "F4",
            "C4",
            "P4",
            "F8",
            "T4",
            "T6",
            "O2",
        ],
        axis=1,
    )
    idx = parquet_data.columns[1:].to_list() + [parquet_data.columns[0]]
    parquet_data = parquet_data[idx].values.T
    parquet_data = butter_lowpass_filter(parquet_data)
    parquet_data = torch.from_numpy(parquet_data).type(torch.float32)
    parquet_data = torch.clip(parquet_data, -1024, 1024)
    return parquet_data

Get the data and labels for training

In [6]:
eeg_data = []
faulty_eeg_id = []
nan_rows = 0

if not os.path.exists(f"{PROCESSED_DATA_DIR}/eeg_labels.pt"):
    for eeg_id, label, offset in zip(train_meta["eeg_id"],all_labels,train_meta["eeg_label_offset_seconds"]):
        parquet_data = pd.read_parquet(INPUT_DATA_DIR + f"/train_eegs/{eeg_id}.parquet").interpolate(method='ffill')[:10000]
        if np.any(parquet_data.isna()):
            faulty_eeg_id.append(eeg_id)
            continue
        
        eeg = extract_parquet(parquet_data)
        eeg_data.append(eeg)
    
    # eeg_data.squeeze()[:,None,:,:].shape
    eeg_data = torch.stack(eeg_data)[:,None,:,:]
    torch.save(eeg_data,f"{PROCESSED_DATA_DIR}/eeg_data.pt")

    all_labels = train_meta.loc[~train_meta["eeg_id"].isin(faulty_eeg_id)][
        ["seizure_vote", "lpd_vote", "gpd_vote", "lrda_vote", "grda_vote", "other_vote"]
    ].values
    eeg_labels = torch.tensor(np.array(all_labels),dtype=torch.float32)
    eeg_labels = eeg_labels/eeg_labels.sum(dim=1,keepdims=True)
    torch.save(eeg_labels,f"{PROCESSED_DATA_DIR}/eeg_labels.pt")
else:
    eeg_data = torch.load(f"{PROCESSED_DATA_DIR}/eeg_data.pt")
    eeg_labels = torch.load(f"{PROCESSED_DATA_DIR}/eeg_labels.pt")

---

# Creating a data loader, preprocessing

Setup global variables for dataloder and preprocessing

In [7]:
SAMPLING_FREQUENCY = 200
SAMPLES_IN_MEASUREMENT = 10000
FOLDS = 5
BATCH_SIZE = 16
NUM_WORKERS = 0

Create a HMS dataset class that will help us load the data during the model training

In [8]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(eeg_data,eeg_labels,test_size=0.1,stratify = eeg_labels.argmax(axis=1))

In [9]:
X_train, y_train = X_train.to(DEVICE), y_train.to(DEVICE)
X_test, y_test = X_test.to(DEVICE), y_test.to(DEVICE)

Create custom dataset and dataloater

In [10]:
from torch.utils.data import DataLoader, Dataset

class CustomImageDataset(Dataset):
    def __init__(self, X,y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_loader = DataLoader(CustomImageDataset(X_train,y_train), batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(CustomImageDataset(X_test,y_test), batch_size=BATCH_SIZE, shuffle=True)

---

# Creating a model

In [11]:
import torch.nn as nn

class SimpleEEGModel(nn.Module):
    def __init__(
        self
    ):
        super(SimpleEEGModel, self).__init__()
        
        self.network = nn.Sequential(
            # input 19 x 10000 x 1 hwc
            # nn.Conv2d(1, 32, (1,3), 1, (0,1), bias=False),
            # nn.BatchNorm2d(32),
            # nn.ReLU(inplace=True),
            # nn.MaxPool2d((1,2)),
            # 
            # # input 19 x 5000 x 32 hwc
            # nn.Conv2d(32, 64, (19,4), 2, 3, bias=False),
            # nn.BatchNorm2d(64),
            # nn.ReLU(inplace=True),
            # nn.MaxPool2d(2),
            # 
            # # input 1 x 2500 x 64 hwc
            # nn.Conv2d(64, 64, (1,4), 1, 3, bias=False),
            # nn.BatchNorm2d(64),
            # nn.ReLU(inplace=True),
            # nn.MaxPool2d(2),
            # 
            # # input 1 x 1250 x 64 hwc
            # nn.Conv2d(64, 64, (1,4), 1, 3, bias=False),
            # nn.BatchNorm2d(64),
            # nn.ReLU(inplace=True),
            # nn.MaxPool2d(2),
            
            # nn.Dropout(0.2),
            
            nn.Flatten(),
            nn.Linear(190, 6),
            nn.Softmax(dim=1)
            
        )
    
    def forward(self, x):
        # print(x[:,:,:,:10])
        out = self.network(x[:,:,:,:10])
        out2 = out - torch.min(out,dim=-1,keepdim=True)[0]
        # for param in self.network.parameters():
        #     print(param.data)
        
        # print(out2)
        # print("========================================")
        return out2/out2.sum(dim=-1,keepdims=True)

# Training (TBD)

---

In [12]:
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.nn import CrossEntropyLoss
from torch.optim import Adam, SGD


model = SimpleEEGModel().to(DEVICE)
optimizer = Adam(model.parameters(), lr=0.0001)
# scheduler = ReduceLROnPlateau(optimizer, 'min')
loss_function = nn.KLDivLoss(reduction="batchmean", log_target=False)

In [13]:
EPOCHS = 10

total_loss=0
for epoch in range(EPOCHS):
    iteration = 0
    for batch in train_loader:
        total_loss = 0
        batch_data = nn.functional.normalize(batch[0],dim=-1)
        batch_labels = batch[1]        
        prediction = model(batch_data)
        loss = loss_function(prediction,batch_labels)
        print(loss.item())
        total_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        iteration+=1
    # scheduler.step()
    print("==========================")
    print(total_loss/iteration)
    print("==========================")

-0.4359705150127411
-0.5342249870300293
-0.6226677894592285
-0.3464440405368805
-0.39145100116729736
-0.31596407294273376
-0.43648141622543335
-0.5220171213150024
-0.44238969683647156
-0.5374261736869812
-0.6701010465621948
-0.6408736705780029
-0.497577965259552
-0.3975502848625183
-0.39829879999160767
-0.37047386169433594
-0.5489099621772766
-0.42310047149658203
-0.38737207651138306
-0.513455867767334
-0.5730209350585938
-0.291964590549469
-0.5180579423904419
-0.2406386435031891
-0.4067748785018921
-0.4184359610080719
-0.3193487524986267
-0.6776068210601807
-0.3433851897716522
-0.3654897212982178
-0.38900038599967957
-0.42537420988082886
-0.27312856912612915
-0.26709914207458496
-0.36506882309913635
-0.3423318564891815
-0.5149547457695007
-0.4254898428916931
-0.5315362215042114
-0.42183709144592285
-0.48968505859375
-0.3768909275531769
-0.41649508476257324
-0.4740526080131531
-0.5089325904846191
-0.4782196283340454
-0.3077821731567383
-0.6165152788162231
-0.2071097046136856
-0.4345004

---